### Actual Code Starts:


In [ ]:
# options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
# driver = webdriver.Firefox(options=options)


# # driver.get('https://www.prokabaddi.com/stats/0-102-player-total-points-statistics') ALL-SEASONS
# driver.get("https://www.prokabaddi.com/stats/44-96-team-total-points-statistics")

# element = driver.find_element(By.CLASS_NAME, "loadmore")
# for i in range(21):
#     # element[0].click()
#     driver.execute_script("arguments[0].click();", element)

# soup = BeautifulSoup(driver.page_source)


# # for item in soup.find_all("div",class_="sipk-lb-playerName"):
# #     print("hello")
# #     [elem.extract() for elem in soup.find("div")]
# #     f = open("parsed.txt", "a")
# #     f.write(item.text)

# #     f.close()
# # playerDetails = pd.read_csv('parsed.txt',encoding='ISO-8859-1',sep='\t',names=['playerName'])
# # print(playerDetails)


# for info in soup.find_all("div", class_="info-name"):

#     first_name_elem = info.find("p", class_="name first-name")
#     last_name_elem = info.find("p", class_="name last-name")

#     if first_name_elem and last_name_elem:
#         first_name = first_name_elem.text.strip()
#         last_name = last_name_elem.text.strip()
#         full_name = f"{first_name} {last_name}"

#         # Write the player's full name to parsed.txt
#         with open("parsed.txt", "a") as f:
#             f.write(full_name + "\n")


# # Load the extracted names into a DataFrame
# playerDetails = pd.read_csv("parsed.txt", encoding="ISO-8859-1", names=["playerName"])
# if os.path.exists("parsed.txt"):
#     os.remove("parsed.txt")


In [ ]:
# print(playerDetails)


In [ ]:
dfObj = pd.DataFrame(data=[])
dfObj["PlayerName"] = playerDetails[playerDetails.columns[0]]
dfObj


In [ ]:
soup = BeautifulSoup(driver.page_source)


for item in soup.find_all("div", class_="table-data matches-played"):
    [elem.extract() for elem in soup.find("div")]
    file = open("parsed.txt", "a")
    if item.text == "Matches Played":
        continue
    file.write(item.text)
    file.write("\n")
    # print(item.text)

    file.close()
playerDetails = pd.read_csv(
    "parsed.txt", encoding="ISO-8859-1", sep="\t", names=["MatchesPlayed"]
)

playerDetails


In [ ]:
playerDetails


In [ ]:
dfObj["MatchesPlayed"] = playerDetails[playerDetails.columns[0]]
dfObj


In [ ]:
if os.path.exists("parsed.txt"):
    os.remove("parsed.txt")


In [ ]:
soup = BeautifulSoup(driver.page_source)
for item in soup.find_all("div", class_="table-data rank"):
    [elem.extract() for elem in soup.find("div")]
    file = open("parsed.txt", "a")

    if item.text == "#":
        continue
    file.write(item.text)
    file.write("\n")

    file.close()
playerDetails = pd.read_csv(
    "parsed.txt", encoding="ISO-8859-1", sep="\t", names=["Rank"]
)


if os.path.exists("parsed.txt"):
    os.remove("parsed.txt")


In [ ]:
dfObj["Rank"] = playerDetails[playerDetails.columns[0]]
dfObj


In [ ]:
soup = BeautifulSoup(driver.page_source)
for item in soup.find_all("div", class_="category"):
    [elem.extract() for elem in soup.find("div")]
    file = open("parsed.txt", "a")
    if item.text:
        file.write(item.text)

    else:
        file.write("NA")
    # file.write(item.text)
    file.write("\n")

    file.close()
playerDetails = pd.read_csv(
    "parsed.txt", encoding="ISO-8859-1", sep="\t", names=["PlayerProfile"]
)

playerDetails


In [ ]:
import os

if os.path.exists("parsed.txt"):
    os.remove("parsed.txt")


In [ ]:
# dfObj["PlayerProfile"] = playerDetails[playerDetails.columns[0]]
# dfObj


In [ ]:
# soup = BeautifulSoup(driver.page_source)
# for item in soup.find_all("div", class_="table-data total-points"):
#     [elem.extract() for elem in soup.find("div")]
#     file = open("parsed.txt", "a")
#     if item.text == "Total Points":
#         continue
#     else:
#         file.write(item.text)

#     file.write("\n")

#     file.close()
# playerDetails = pd.read_csv(
#     "parsed.txt", encoding="ISO-8859-1", sep="\t", names=["TotalPoints"]
# )
# print(playerDetails)

# import os

# if os.path.exists("parsed.txt"):
#     os.remove("parsed.txt")


In [ ]:
# dfObj["TotalPoints"] = playerDetails[playerDetails.columns[0]]
# dfObj


In [ ]:
# dfObj.describe()


In [ ]:
# dfObj.to_csv("season10_total-points-statistics.csv")


## terrible code is above. now writing better code


In [3]:
import pandas as pd
import json
import re


def extract_json_from_script(script_content):
    # Find the JSON data within the script
    start = script_content.find("window.statsWidgetData = ")
    if start == -1:
        return None
    start += len("window.statsWidgetData = ")

    # Find the end of the JSON object
    bracket_count = 0
    end = start
    for i, char in enumerate(script_content[start:]):
        if char == "{":
            bracket_count += 1
        elif char == "}":
            bracket_count -= 1
            if bracket_count == 0:
                end = start + i + 1
                break

    if bracket_count != 0:
        return None

    json_str = script_content[start:end]

    # Parse the JSON data
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None


# Extract JSON data from the script content
json_data = extract_json_from_script(script_content)
if json_data is None:
    print("Error: Could not extract JSON data from the file.")
    exit(1)

# Extract the team data
team_data = json_data["stats"]["data"]

# Create a DataFrame
df = pd.DataFrame(team_data)

# Convert 'value' and 'match_played' to numeric type
df["value"] = pd.to_numeric(df["value"])
df["match_played"] = pd.to_numeric(df["match_played"])

# Calculate points per match
df["points_per_match"] = df["value"] / df["match_played"]

# Display basic information about the DataFrame
print(df.info())

# Display the first few rows
print(df.head())

# Basic statistics
print(df.describe())

# Top teams by total points
top_teams = df.sort_values("value", ascending=False)
print("\nTeams ranked by total points:")
print(top_teams[["team_name", "value", "match_played"]])

# Top 5 teams by points per match
top_scoring_teams = df.sort_values("points_per_match", ascending=False).head(5)
print("\nTop 5 teams by points per match:")
print(top_scoring_teams[["team_name", "points_per_match", "match_played", "value"]])

# Correlation between matches played and total points
correlation = df["match_played"].corr(df["value"])
print(f"\nCorrelation between matches played and total points: {correlation:.2f}")

# Calculate the average points per match for all teams
avg_points_per_match = df["points_per_match"].mean()
print(f"\nAverage points per match across all teams: {avg_points_per_match:.2f}")

# Teams above and below average
above_average = df[df["points_per_match"] > avg_points_per_match]
below_average = df[df["points_per_match"] <= avg_points_per_match]

print("\nTeams performing above average:")
print(above_average[["team_name", "points_per_match"]])

print("\nTeams performing below average:")
print(below_average[["team_name", "points_per_match"]])

# Save the DataFrame to a CSV file
df.to_csv("season10_team_total_points_statistics.csv", index=False)
print("\nData has been saved to 'season10_team_total_points_statistics.csv'")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_played      12 non-null     int64  
 1   value             12 non-null     int64  
 2   team_id           12 non-null     int64  
 3   team_name         12 non-null     object 
 4   rank              12 non-null     object 
 5   points_per_match  12 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 704.0+ bytes
None
   match_played  value  team_id             team_name rank  points_per_match
0            24    950        7         Puneri Paltan    1         39.583333
1            25    920       28      Haryana Steelers    2         36.800000
2            24    874        6         Patna Pirates    3         36.416667
3            23    843        3  Jaipur Pink Panthers    4         36.652174
4            22    832       29       Tamil Thalaivas    5       

In [ ]:
import pandas as pd
import json
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time


def extract_json_from_script(script_content):
    # Find the JSON data within the script
    start = script_content.find("window.statsWidgetData = ")
    if start == -1:
        return None
    start += len("window.statsWidgetData = ")

    # Find the end of the JSON object
    bracket_count = 0
    end = start
    for i, char in enumerate(script_content[start:]):
        if char == "{":
            bracket_count += 1
        elif char == "}":
            bracket_count -= 1
            if bracket_count == 0:
                end = start + i + 1
                break

    if bracket_count != 0:
        return None

    json_str = script_content[start:end]

    # Parse the JSON data
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None


def fetch_and_process_data(url):
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(url)

        # # Click the 'loadmore' button multiple times
        # for i in range(21):
        #     try:
        #         element = WebDriverWait(driver, 10).until(
        #             EC.presence_of_element_located((By.CLASS_NAME, 'loadmore'))
        #         )
        #         driver.execute_script("arguments[0].click();", element)
        #         time.sleep(1)  # Wait for content to load
        #     except:
        #         print(f"Stopped after {i} clicks")
        #         break

        # Extract the script content
        soup = BeautifulSoup(driver.page_source)
        # print(soup)
        script_tag = soup.find(
            "script", string=lambda text: text and "window.statsWidgetData" in text
        )

        # print(script_tag)

        # script_content = script_tag

        # json_data = extract_json_from_script(script_content)

        if script_tag:
            json_data = extract_json_from_script(script_tag.string)
            if json_data:
                player_data = json_data["stats"]["data"]
                df = pd.DataFrame(player_data)

                # Convert 'value' and 'match_played' to numeric type
                df["value"] = pd.to_numeric(df["value"])
                df["match_played"] = pd.to_numeric(df["match_played"])

                # Calculate points per match
                df["points_per_match"] = df["value"] / df["match_played"]

                return df

    finally:
        driver.quit()

    return None


# URL of the webpage
url = "https://www.prokabaddi.com/stats/44-22-raid-points-statistics"

# Fetch and process data
df = fetch_and_process_data(url)

if df is not None:
    # Display basic information about the DataFrame
    print(df.info())

    # Display the first few rows
    print(df.head())

    # Basic statistics
    print(df.describe())

    # Top 10 players by total points
    top_10_players = df.sort_values("value", ascending=False).head(10)
    print("\nTop 10 players by total points:")
    print(top_10_players[["player_name", "value", "team_name"]])

    # Top 5 players by points per match (minimum 10 matches played)
    top_players = (
        df[df["match_played"] >= 10]
        .sort_values("points_per_match", ascending=False)
        .head(5)
    )
    print("\nTop 5 players by points per match (minimum 10 matches):")
    print(top_players[["player_name", "points_per_match", "match_played", "team_name"]])

    # Team performance
    team_performance = (
        df.groupby("team_name")["value"].sum().sort_values(ascending=False)
    )
    print("\nTeam performance (total points):")
    print(team_performance)

    # Position-wise analysis
    position_analysis = df.groupby("position_name")["value"].agg(
        ["sum", "mean", "count"]
    )
    print("\nPosition-wise analysis:")
    print(position_analysis)

    # Correlation between matches played and total points
    correlation = df["match_played"].corr(df["value"])
    print(f"\nCorrelation between matches played and total points: {correlation:.2f}")
else:
    print("Failed to fetch or process data from the webpage.")

df.to_csv("season10_raid-points-statistics.csv")


# this is the main script, run this!!!!!, everything else was building up to this!


In [7]:
import pandas as pd
import json
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time

d = {
    "s-10": "44",
    "s-9": "25",
    "s-8": "20",
    "s-7": "11",
    "s-6": "10",
    "s-5": "8",
    "s-4": "4",
    "s-3": "3",
    "s-2": "2",
    "s-1": "1",
    "s-all": "0",
}


def extract_json_from_script(script_content):
    start = script_content.find("window.statsWidgetData = ")
    if start == -1:
        return None
    start += len("window.statsWidgetData = ")

    bracket_count = 0
    end = start
    for i, char in enumerate(script_content[start:]):
        if char == "{":
            bracket_count += 1
        elif char == "}":
            bracket_count -= 1
            if bracket_count == 0:
                end = start + i + 1
                break

    if bracket_count != 0:
        return None

    json_str = script_content[start:end]

    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None


def fetch_and_process_data(url):
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(url)

        # We don't need to click "Load More" for team stats, as all teams are usually shown on one page

        soup = BeautifulSoup(driver.page_source, "html.parser")
        script_tag = soup.find(
            "script", string=lambda text: text and "window.statsWidgetData" in text
        )

        if script_tag:
            json_data = extract_json_from_script(script_tag.string)
            if json_data:
                team_data = json_data["stats"]["data"]
                df = pd.DataFrame(team_data)

                df["value"] = pd.to_numeric(df["value"])
                df["match_played"] = pd.to_numeric(df["match_played"])

                df["points_per_match"] = df["value"] / df["match_played"]

                return df, json_data["stats"]["stat"]

    finally:
        driver.quit()

    return None, None


urls = [
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-96-total-points-scored-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-14-team-raid-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-17-team-successful-raid-percent-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-18-team-successful-tackles-per-match-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-19-team-successful-tackle-percent-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-133-total-points-conceded-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-140-avg-points-scored-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-13-successful-raids-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-97-raid-points-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-98-avg-raid-points-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-15-successful-tackles-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-95-tackle-points-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-99-avg-tackle-points-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-134-super-raid-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-20-super-tackles-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-135-do-or-die-raid-points-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-136-all-outs-inflicted-statistics',
    f'https://www.prokabaddi.com/stats/{d["s-10"]}-137-all-outs-conceded-statistics',
]

x = 0
for url in urls:
    x += 1
    df, stat_name = fetch_and_process_data(url)
    if df is not None:
        # Create a file name based on the statistic
        file_name = f"{x}_df_team_season10_{stat_name.lower().replace(' ', '-')}.csv"

        # Export the DataFrame to CSV
        df.to_csv(file_name, index=False)
        print(f"Exported {file_name}")

        # Display basic information about the DataFrame
        print(f"\nStatistic: {stat_name}")
        print(df.info())

        # Display the first few rows
        print(df.head())

        # Basic statistics
        print(df.describe())

        # Team rankings
        team_rankings = df.sort_values("value", ascending=False)
        print(f"\nTeam rankings by {stat_name}:")
        print(team_rankings[["team_name", "value", "match_played"]])

        # Top 5 teams by points per match
        top_teams_per_match = df.sort_values("points_per_match", ascending=False).head(
            5
        )
        print(f"\nTop 5 teams by {stat_name} per match:")
        print(
            top_teams_per_match[
                ["team_name", "points_per_match", "match_played", "value"]
            ]
        )

        # Correlation between matches played and the statistic
        correlation = df["match_played"].corr(df["value"])
        print(
            f"\nCorrelation between matches played and {stat_name}: {correlation:.2f}"
        )

        # Calculate league average
        league_avg = df["points_per_match"].mean()
        print(f"\nLeague average {stat_name} per match: {league_avg:.2f}")

        # Teams above and below league average
        above_avg = df[df["points_per_match"] > league_avg]
        below_avg = df[df["points_per_match"] <= league_avg]

        print("\nTeams above league average:")
        print(above_avg[["team_name", "points_per_match"]])

        print("\nTeams below league average:")
        print(below_avg[["team_name", "points_per_match"]])

        print("\n" + "=" * 50 + "\n")
    else:
        print(f"Failed to fetch or process data from {url}")


JSON data saved to data.json


In [11]:
import os
import pandas as pd
import json
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time

    # "s-9": "25", 
    # "s-8": "20",
    # "s-10": "44", 

d = {

    "s-7": "11",
    "s-6": "10",
    "s-5": "8",
    "s-4": "4",
    "s-3": "3",
    "s-2": "2",
    "s-1": "1",
    "s-all": "0",
}

for k, v in d.items():

    def extract_json_from_script(script_content):
        start = script_content.find('window.statsWidgetData = ')
        if start == -1:
            return None
        start += len('window.statsWidgetData = ')
        
        bracket_count = 0
        end = start
        for i, char in enumerate(script_content[start:]):
            if char == '{':
                bracket_count += 1
            elif char == '}':
                bracket_count -= 1
                if bracket_count == 0:
                    end = start + i + 1
                    break
        
        if bracket_count != 0:
            return None
        
        json_str = script_content[start:end]
        
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return None

    def fetch_and_process_data(url):
        options = webdriver.FirefoxOptions()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options)
        
        try:
            driver.get(url)
            
            # We don't need to click "Load More" for team stats, as all teams are usually shown on one page
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            script_tag = soup.find('script', string=lambda text: text and 'window.statsWidgetData' in text)
            
            if script_tag:
                json_data = extract_json_from_script(script_tag.string)
                if json_data:
                    try:
                        team_data = json_data['stats']['data']
                        df = pd.DataFrame(team_data)
                        
                        df['value'] = pd.to_numeric(df['value'])
                        df['match_played'] = pd.to_numeric(df['match_played'])
                        
                        df['points_per_match'] = df['value'] / df['match_played']
                        
                        return df, json_data['stats']['stat']
                    
                    except KeyError as e:
                        print(f"KeyError: {e} - The 'stats' key is missing or malformed in the data from {url}. Skipping this URL.")
                        return None, None
        
        except Exception as e:
            print(f"An error occurred while processing the URL {url}: {str(e)}")
        
        finally:
            driver.quit()
        
        return None, None


    urls = [
        f'https://www.prokabaddi.com/stats/{d[k]}-96-total-points-scored-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-14-team-raid-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-17-team-successful-raid-percent-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-18-team-successful-tackles-per-match-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-19-team-successful-tackle-percent-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-133-total-points-conceded-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-140-avg-points-scored-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-13-successful-raids-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-97-raid-points-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-98-avg-raid-points-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-15-successful-tackles-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-95-tackle-points-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-99-avg-tackle-points-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-134-super-raid-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-20-super-tackles-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-135-do-or-die-raid-points-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-136-all-outs-inflicted-statistics',
        f'https://www.prokabaddi.com/stats/{d[k]}-137-all-outs-conceded-statistics'
    ]   

    # Create a directory for the season if it doesn't exist
    season_dir = f"season_{k}"
    if not os.path.exists(season_dir):
        os.makedirs(season_dir)

    x = 0
    for url in urls:
        x += 1
        df, stat_name = fetch_and_process_data(url)
        if df is not None:
            # Create a file name based on the statistic
            file_name = f"{x}_df_team_season{k}_{stat_name.lower().replace(' ', '-')}.csv"
            
            # Define the full path for the CSV file
            file_path = os.path.join(season_dir, file_name)
            
            # Export the DataFrame to CSV
            df.to_csv(file_path, index=False)
            print(f"Exported {file_path}")
            
            # Display basic information about the DataFrame
            print(f"\nStatistic: {stat_name}")
            print(df.info())
            
            # Display the first few rows
            print(df.head())
            
            # Basic statistics
            print(df.describe())
            
            # Team rankings
            team_rankings = df.sort_values('value', ascending=False)
            print(f"\nTeam rankings by {stat_name}:")
            print(team_rankings[['team_name', 'value', 'match_played']])
            
            # Top 5 teams by points per match
            top_teams_per_match = df.sort_values('points_per_match', ascending=False).head(5)
            print(f"\nTop 5 teams by {stat_name} per match:")
            print(top_teams_per_match[['team_name', 'points_per_match', 'match_played', 'value']])
            
            # Correlation between matches played and the statistic
            correlation = df['match_played'].corr(df['value'])
            print(f"\nCorrelation between matches played and {stat_name}: {correlation:.2f}")
            
            # Calculate league average
            league_avg = df['points_per_match'].mean()
            print(f"\nLeague average {stat_name} per match: {league_avg:.2f}")
            
            # Teams above and below league average
            above_avg = df[df['points_per_match'] > league_avg]
            below_avg = df[df['points_per_match'] <= league_avg]
            
            print("\nTeams above league average:")
            print(above_avg[['team_name', 'points_per_match']])
            
            print("\nTeams below league average:")
            print(below_avg[['team_name', 'points_per_match']])
            
            print("\n" + "="*50 + "\n")
        else:
            print(f"Failed to fetch or process data from {url}")


Exported season_s-7/1_df_team_seasons-7_team-total-points.csv

Statistic: Team Total Points
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   match_played      12 non-null     int64  
 1   value             12 non-null     int64  
 2   team_id           12 non-null     int64  
 3   team_name         12 non-null     object 
 4   rank              12 non-null     object 
 5   points_per_match  12 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 704.0+ bytes
None
   match_played  value  team_id          team_name rank  points_per_match
0            24    890        2  Dabang Delhi K.C.    1         37.083333
1            24    868        4    Bengal Warriors    2         36.166667
2            24    868        1    Bengaluru Bulls    3         36.166667
3            23    809       28   Haryana Steelers    4       